In [4]:
import torch
!pip install torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.autograd import Function
!pip install python-resize-image
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from pylab import *
from resizeimage import resizeimage
import os
import time

In [2]:
import torch.optim

In [3]:
#create data
root_dir="mnist_png"
transform2=transforms.Compose([transforms.Grayscale(1),transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_data=ImageFolder(root=os.path.join(root_dir,"training"),transform=transform2)
test_data=ImageFolder(root=os.path.join(root_dir,"testing"),transform=transform2)
print(train_data.classes,test_data.classes)
batch_size=50
train11_loader=torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test11_loader =torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'tout'] ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'tout']


In [4]:
class RN(torch.nn.Module):
    def __init__(self,nbHiddenLayers,nbNeuronsPerLayer,n_in=28*28,n_out=11,): #3 couches cachees
        super().__init__()
        #fully connected layer
        self.dim_in = n_in
        self.dim_out = n_out
        self.n_hidden_layers=nbHiddenLayers
        self.nbNPerLayer=nbNeuronsPerLayer
        #self.fc=None
        if(self.n_hidden_layers > 0):
            self.fc1 = torch.nn.Linear(self.dim_in,self.nbNPerLayer,bias=True)# In -> first hidden
             # Hidden -> hidden
            self.fc2=torch.nn.Linear(self.nbNPerLayer,self.nbNPerLayer,bias=True)
            self.fc3=torch.nn.Linear(self.nbNPerLayer,self.dim_out,bias=True) # -> last hidden -> out
        else:
            self.fc1 = [torch.nn.Linear(self.dim_in,self.dim_out,bias=True)] # Single-layer perceptron
    
    def forward(self, x):
        fc=[self.fc1,self.fc2,self.fc3]
        #fc=[self.fc1,self.fc2,self.fc3,self.fc4]
        for i in range(self.n_hidden_layers):
            x=F.relu(fc[i](x))
        x=fc[-1](x)
        #return F.log_softmax(x,dim=1)
        return F.softmax(x,dim=1)

In [5]:

def train(nbEntr,NN,loader,lr=1e-3):
    start=time.time()
    optimizer=torch.optim.Adam(NN.parameters(),lr) #lr : learning rate
    lloss=[]
    for i in range(nbEntr):
        for x,y in loader:
            NN.zero_grad() #
            output=NN(x.view(-1,28*28))
            loss=F.cross_entropy(output,y)
            #loss=F.nll_loss(output,y)
            loss.backward()
            optimizer.step()
        lloss.append(loss)
    end=time.time()
    print("Time execution of ", nbEntr ," trainings of ",end-start," s" )
    return lloss


def test(loader,NN):
    start=time.time()
    correct_class =torch.zeros((11))
    total=torch.zeros((11))
    correct=0
    with torch.no_grad():
        for x,y in loader:
            output=NN(x.view(-1,28*28))
            for i,img in enumerate(output):
                if(torch.argmax(img)==y[i]):
                    correct+=1
                    correct_class[y[i]]+=1
                total[y[i]]+=1
    end=time.time()
    print("Time execution :  ",end-start)
    return correct_class,total,correct/total.sum()

def odd(correct,total):
    res= []
    for i in range(len(total)):
        res.append(correct[i]/total[i])
    return res

In [6]:
#50 -> 0.37
train0=ImageFolder(root="testN",transform=transform2)
loader0=torch.utils.data.DataLoader(train0, batch_size=1, shuffle=True)

In [7]:
imgCam=ImageFolder(root="testC",transform=transform2)
imgCam_loader =torch.utils.data.DataLoader(imgCam, batch_size=batch_size, shuffle=False)

In [100]:
nbHiddenLayers = 2
nbNeuronsPerLayer = 30
myNN=RN(nbHiddenLayers,nbNeuronsPerLayer)

In [85]:
train(5,myNN,loader0,lr=1e-3)

Time execution of  5  trainings of  0.33475160598754883  s


[tensor(2.3916, grad_fn=<NllLossBackward>),
 tensor(2.3701, grad_fn=<NllLossBackward>),
 tensor(2.3624, grad_fn=<NllLossBackward>),
 tensor(2.3905, grad_fn=<NllLossBackward>),
 tensor(2.3301, grad_fn=<NllLossBackward>)]

In [86]:
c,t,s=test(loader0,myNN)

Time execution :   0.017626285552978516


In [87]:
print(s)
print(odd(c,t))

tensor(1.)
[tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(nan)]


In [16]:
ct,tt,st=test(test11_loader,myNN)

Time execution :   11.959380865097046


In [17]:
print(st)
print(odd(ct,tt))

tensor(0.1242)
[tensor(0.9122), tensor(0.), tensor(0.), tensor(0.0248), tensor(0.), tensor(0.2321), tensor(0.), tensor(0.), tensor(0.1828), tensor(0.), tensor(0.)]


In [29]:
trans = transforms.Compose([transforms.ToTensor()])
demo = Image.open("testC/0/a0.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]

tensor([2.8115e-03, 2.9376e-03, 3.2019e-02, 5.3911e-03, 1.3177e-09, 8.2396e-01,
        2.8342e-03, 2.7236e-03, 1.2923e-02, 3.2776e-05, 1.1437e-01],
       grad_fn=<SelectBackward>)

In [90]:
trans = transforms.Compose([transforms.ToTensor()])
demo = Image.open("testC/0/a0.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]


tensor([7.3637e-01, 1.2483e-02, 1.6958e-03, 6.8195e-04, 7.9221e-06, 7.2840e-03,
        5.8710e-03, 2.2175e-02, 2.4016e-04, 7.7452e-05, 2.1312e-01],
       grad_fn=<SelectBackward>)

In [104]:
#image camera
trans = transforms.Compose([transforms.ToTensor()])
#demo = Image.open("testC/1/ft1_425.png") 
demo = Image.open("testC/5/ft5_444.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]


tensor([0.0861, 0.1079, 0.0759, 0.0939, 0.0895, 0.1109, 0.0822, 0.0835, 0.0977,
        0.0819, 0.0906], grad_fn=<SelectBackward>)

In [92]:
#image camera
trans = transforms.Compose([transforms.ToTensor()])
demo = Image.open("testC/1/ft1_67.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]

tensor([1.1445e-08, 2.7946e-10, 1.4167e-17, 2.1458e-17, 7.6985e-18, 2.6081e-14,
        4.3537e-12, 1.8772e-11, 3.6164e-17, 1.4232e-15, 1.0000e+00],
       grad_fn=<SelectBackward>)

In [99]:
#image camera
trans = transforms.Compose([transforms.ToTensor()])
demo = Image.open("testC/8/ft8_67.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]

tensor([4.3677e-09, 6.2508e-11, 4.1629e-19, 2.1297e-18, 1.8690e-19, 1.1170e-13,
        2.6319e-12, 1.6533e-12, 3.9851e-18, 7.9422e-16, 1.0000e+00],
       grad_fn=<SelectBackward>)

In [94]:
#image camera
trans = transforms.Compose([transforms.ToTensor()])
demo = Image.open("mnist_png/testing/0/69.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]

tensor([9.9879e-01, 1.0629e-05, 2.4153e-05, 1.2297e-07, 2.2967e-11, 9.3075e-05,
        1.9679e-04, 1.2917e-04, 3.9680e-07, 9.4948e-09, 7.5951e-04],
       grad_fn=<SelectBackward>)

In [95]:
#image camera
trans = transforms.Compose([transforms.ToTensor()])
demo = Image.open("mnist_png/testing/1/39.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]

tensor([2.9233e-06, 9.9347e-01, 3.4442e-05, 1.3838e-03, 2.8973e-06, 1.4669e-04,
        1.1764e-03, 7.9726e-06, 1.0449e-03, 3.2773e-06, 2.7315e-03],
       grad_fn=<SelectBackward>)

In [25]:
trans = transforms.Compose([transforms.ToTensor()])
demo = Image.open("mnist_png/testing/2/905.png") 
demo_img = trans(demo)
myNN(demo_img.view(-1,28*28))[0]

tensor([0.1191, 0.0744, 0.1030, 0.0763, 0.1129, 0.0772, 0.1280, 0.0904, 0.0612,
        0.0923, 0.0651], grad_fn=<SelectBackward>)

In [88]:
#nbHiddenLayers = 
#nbNeuronsPerLayer = 15
#myNN=RN(nbHiddenLayers,nbNeuronsPerLayer)
train(5,myNN,train11_loader,lr=1e-3)
c,t,s=test(train11_loader,myNN)
print(s)
print(odd(c,t))

Time execution of  5  trainings of  494.81303691864014  s
Time execution :   112.87179493904114
tensor(0.9643)
[tensor(0.9829), tensor(0.9869), tensor(0.9491), tensor(0.9551), tensor(0.9577), tensor(0.9598), tensor(0.9863), tensor(0.9694), tensor(0.9363), tensor(0.9509), tensor(0.9950)]


In [89]:
ct,tt,st=test(test11_loader,myNN)
print(st)
print(odd(ct,tt))

Time execution :   20.474367380142212
tensor(0.9580)
[tensor(0.9806), tensor(0.9921), tensor(0.9341), tensor(0.9644), tensor(0.9542), tensor(0.9451), tensor(0.9729), tensor(0.9562), tensor(0.9251), tensor(0.9376), tensor(0.9840)]


In [ ]:
#image de testC
for i in imgCam_loader:
    x,y=i
    output=myNN(x.view(-1,28*28))
    for i,img in enumerate(output):
        plt.imshow(x[i].view(28,28))
        plt.title("Predicted class {}".format(torch.argmax(myNN(x[i].view(-1,28*28))[0])))
        plt.show()
        print((myNN(x[i].view(-1,28*28))[0]))
        break